<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


# Part 4: SpaceX Launches — Data Exploration with SQL

**Short description:**  
This notebook loads a SpaceX CSV into a local SQLite database, creates a SQL table, and runs a sequence of SQL queries (SELECT DISTINCT, LIKE, SUM, AVG, MIN, BETWEEN, GROUP BY, subqueries, substring filters, ordering) to explore and summarize launch data.

**Dataset link:**

Click on the link below to download and save the dataset (.CSV file):

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>

**Objectives**
- Load a CSV dataset into an SQLite database from a pandas DataFrame.
- Create and query SQL tables using SQLite via Python (`sqlite3`) and `pandas.read_sql_query`.
- Practice common SQL operations: DISTINCT, WHERE with LIKE, aggregate functions (SUM, AVG, MIN), BETWEEN, GROUP BY, subqueries and substring functions.
- Illustrate how to integrate SQL queries into a data science workflow (inspect results as DataFrames and print outputs).
- Provide reproducible steps and documentation so reviewers can re-run the queries locally.

**Notice about documentation:**  
The original notebook submission (course assignment) was kept intact. I have **only modified documentation (comments, headings, markdown)** and made **minimal, necessary corrections** to ensure the notebook runs without errors. All rights related to the lab/workshop design and original exercise belong exclusively to **IBM Corporation**. This notebook includes additional documentation for clarity, but the intellectual property of the original exercise is retained by IBM.

---

## Table of contents

1. Dependencies & execution instructions  
2. Load CSV & create local SQLite DB (to_sql)  
3. Create persistent table with filtering (CREATE TABLE AS)  
4. SQL queries (examples 1–12) — explanation & execute  
5. Notes & reproducibility

## 1) Dependencies & execution instructions

This section installs and imports required Python packages.  

**Recommended local execution steps:**

1. Create and activate a Python virtual environment:
   - `python -m venv venv`
   - `source venv/bin/activate` (macOS / Linux) or `venv\Scripts\activate` (Windows)
2. Install dependencies:
   - `pip install -r requirements.txt`
3. Launch Jupyter Notebook:
   - `jupyter notebook`
4. Open this notebook and run cells top-to-bottom. The SQLite database (`my_data1.db`) will be created in the notebook's working directory.

In [1]:
import pandas as pd
import sqlite3

## 2) Load CSV & create local SQLite database

This section reads the SpaceX CSV into a pandas DataFrame and writes it to an SQLite table (`SPACEXTBL`) using `DataFrame.to_sql`. 

In [2]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [3]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

## 3) Create persistent table with filtering (CREATE TABLE AS)

This section demonstrates creating a new SQL table (`SPACEXTABLE`) from `SPACEXTBL` while filtering out rows with NULL `Date`. This is done with a `CREATE TABLE AS SELECT` SQL statement executed via the SQLite connection.

In [4]:
query = "CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;"
con.execute(query)

## 4) SQL queries (examples 1–12)

This section runs a series of SQL queries and prints the results. Queries include:
1. `SELECT DISTINCT` to list distinct launch sites.
2. `LIKE` with `LIMIT`.
3. `SUM` for a customer group (e.g., NASA (CRS)).
4. `AVG` for payload mass by booster version.
5. `MIN` to find earliest date for a landing outcome.
6. `BETWEEN` to filter by payload mass and landing outcome.
7. `GROUP BY` to count mission outcomes.
8. `SUBQUERY` for MAX payload mass accessor.
9. `SUBSTR` usage for string-based date filtering.
10. `BETWEEN + GROUP BY + ORDER` for counting landing outcomes in a date range.
11. Example query with alternative date string format to illustrate possible pitfalls.
Each query is executed with `pd.read_sql_query` and printed as a DataFrame for inspection.

### 4.1 Names of the unique launch sites in the space mission


In [5]:
query = "SELECT DISTINCT(Launch_Site) FROM SPACEXTBL;"
df_result = pd.read_sql_query(query, con)
df_result

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


### 4.2 5 records where launch sites begin with the string 'CCA' 


In [6]:
query = "SELECT * FROM SPACEXTBL WHERE LAUNCH_SITE LIKE 'CCA%' LIMIT 5;"
df_result = pd.read_sql_query(query, con)
df_result

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### 4.3 Total payload mass carried by boosters launched by NASA (CRS)


In [7]:
query = "SELECT SUM(PAYLOAD_MASS__KG_) FROM SPACEXTBL WHERE CUSTOMER == 'NASA (CRS)';"
df_result = pd.read_sql_query(query, con)
df_result

,SUM(PAYLOAD_MASS__KG_)
0,45596


### 4.4 Average payload mass carried by booster version F9 v1.1


In [8]:
query = "SELECT AVG(PAYLOAD_MASS__KG_) FROM SPACEXTBL WHERE BOOSTER_VERSION = 'F9 v1.1';"
df_result = pd.read_sql_query(query, con)
df_result

,AVG(PAYLOAD_MASS__KG_)
0,2928.4


### 4.5 Date when the first succesful landing outcome in ground pad was acheived

In [9]:
query = "SELECT MIN(DATE) FROM SPACEXTBL WHERE LANDING_OUTCOME  = 'Success (ground pad)';"
df_result = pd.read_sql_query(query, con)
df_result

,MIN(DATE)
0,2015-12-22


### Task 6

### 4.6 Names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [10]:
query = """
SELECT BOOSTER_VERSION
FROM SPACEXTBL
WHERE LANDING_OUTCOME = 'Success (drone ship)'
AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;
"""
df_result = pd.read_sql_query(query, con)
df_result

,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


### 4.7 Total number of successful and failure mission outcomes


In [11]:
query = "SELECT MISSION_OUTCOME, COUNT(*) as total FROM SPACEXTBL GROUP BY MISSION_OUTCOME;"
df_result = pd.read_sql_query(query, con)
df_result

,Mission_Outcome,total
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1


### 4.8 Names of the booster_versions which have carried the maximum payload mass


In [12]:
query = """
SELECT BOOSTER_VERSION
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ IN (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL);
"""
df_result = pd.read_sql_query(query, con)
df_result

,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


### 4.9 Records which will display the month names, failure landing_outcomes in drone ship, booster_versions, launch_site for the months in year 2015.


In [13]:
query = """
SELECT substr(Date, 6,2) AS MONTH, DATE, LANDING_OUTCOME, BOOSTER_VERSION, LAUNCH_SITE
FROM SPACEXTBL
WHERE substr(Date,0,5) = '2015'
AND LANDING_OUTCOME = 'Failure (drone ship)';
"""
df_result = pd.read_sql_query(query, con)
df_result

,MONTH,Date,Landing_Outcome,Booster_Version,Launch_Site
0,01,2015-01-10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,04,2015-04-14,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### 4.10 Rank of the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [14]:
query = """
SELECT LANDING_OUTCOME, COUNT(*) as COUNT
FROM SPACEXTBL
WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY LANDING_OUTCOME
ORDER BY COUNT DESC;
"""
df_result = pd.read_sql_query(query, con)
df_result

,Landing_Outcome,COUNT
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1


In [15]:
query = """
SELECT LANDING_OUTCOME, COUNT(*) as COUNT
FROM SPACEXTBL
WHERE DATE BETWEEN '04-06-2010' and '20-03-2017'
GROUP BY LANDING_OUTCOME
ORDER BY COUNT DESC;
"""
df_result = pd.read_sql_query(query, con)
df_result

,Landing_Outcome,COUNT


In [16]:
con.close()

## 5) Notes & reproducibility

- The created database file (`my_data1.db`) will be created in the working directory.
- Date string formats can vary — ensure your SQL filters match the dataset date format or convert dates to proper SQL date types if needed.


## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
